In [1]:
from torchtext.data import Field, TabularDataset, BucketIterator

1. specify how preprocessing should be done -> `Fields`
2. use dataset to load the data -> `TabularDataset` (JSON,csv,tsv)
3. Construct an iterator to do batch, padding -> `BucketIterator`

In [2]:
# show data
# preprocessing is done
import pandas as pd
train = pd.read_csv('review_sampled_train.tsv', delimiter='\t', header=None)
train.head()

,0,1
0,negative,주문 한 제품 잘못 보내 서 맞교환 해 준다고 해 놓 고 제품 수거 만 해 가 고 ...
1,negative,박스 안 에 아무런 포장 없이 제품 동봉 후 배송 됨 . 아무리 플라스틱 소재 병 ...
2,negative,안 밝 습니다
3,negative,생각 보다 가방 이 무지 크 네요 . .. 빨간 가방 인데 먼가 검 은 게 묻 어 ...
4,negative,"19 일 주문건 구매 확정 요구 이메일 은 어제 받 았 는데 , 앵글 은 오늘 도착..."


In [3]:
# Field

label = Field(
    sequential=False,
    use_vocab=True,
    unk_token=None,
    is_target=True
)
use_eos = False
text = Field(
    use_vocab=True,
    batch_first=True,
    include_lengths=False,
    eos_token='<EOS>' if use_eos else None
)

In [4]:
# TabularDataset

valid_ratio = 0.2

train, valid = TabularDataset(
    path='./review_sampled_train.tsv',
    format='tsv', 
    fields=[
        ('label', label),
        ('text', text),
    ],
).split(split_ratio=(1 - valid_ratio))

In [5]:
# build vocab

max_size = 9999
min_freq = 5

label.build_vocab(train)
text.build_vocab(train, max_size=max_size, min_freq=min_freq)

In [6]:
# BucketIterator
batch_size = 128
device = -1
shuffle = True

train_loader, valid_loader = BucketIterator.splits(
    (train, valid),
    batch_size=batch_size,
    device='cuda:%d' % device if device >= 0 else 'cpu',
    shuffle=shuffle,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

In [10]:
for (x_i, y_i) in train_loader:
    print(x_i.shape)
    print(y_i.shape)
    break

torch.Size([128, 16])
torch.Size([128])


# Done!

In [12]:
print("|train|=%d" % len(train_loader.dataset))
print("|valid|=%d" % len(valid_loader.dataset))
print("|vocab|=%d" % len(text.vocab))
print("|label|=%d" % len(label.vocab))

|train|=8000
|valid|=2000
|vocab|=3039
|label|=2


In [17]:
text.vocab.itos[100]

'수'

In [18]:
text.vocab.stoi['배송']

18

In [22]:
data = next(iter(train_loader))

print(data.text.shape)
print(data.label.shape)

torch.Size([128, 8])
torch.Size([128])


In [25]:
data.text[0]

tensor([ 454,    7,  108,    3, 1998,    7,    0,   29])

In [27]:
line = []
for x_i in data.text[0]:
    line += [text.vocab.itos[x_i]]
    
print(' '.join(line))

색깔 도 괜찮 고 기장 도 <unk> 합니다
